# EasyUQ on precipitation example

    1. Conformal prediction 
    2. EasyUQ
    3. Single gaussian
    4. Ensemble data

In [1]:
import numpy as np
import isodisreg 
from isodisreg import idr
from helper_functions import algo72_ensemble, log_norm_optim, crps_censored_gaussian
import properscoring as ps
import pandas as pd

In [2]:
# lead_time: 1, 2, 3, 4, 5 (in days)
lead_time = 1 

In [ ]:
fra_data = np.loadtxt('./data/fra_lead_'+str(lead_time)+'.txt')
dim_train = 2896
dim_test = 721
dim_train2 = dim_train-dim_test
y = fra_data[:, 0]
y_train = y[0:dim_train]
y_test = y[dim_train:]

hres = fra_data[:, 1]
hres_train = hres[0:dim_train]
hres_test = hres[dim_train:]

In [ ]:
# 1. Conformal prediction

In [5]:
ens_cp = algo72_ensemble(hres_train, hres_test, y_train)
ens_cp0 = np.where(ens_cp < 0, 0, ens_cp)

print('CRPS CP ensemble:' , np.round(np.mean(ps.crps_ensemble(y_test,ens_cp)), 3))
print('CRPS zero-censored CP ensemble:', np.round(np.mean(ps.crps_ensemble(y_test,ens_cp0)),3))

CRPS CP ensemble: 0.886
CRPS zero-censored CP ensemble: 0.85


In [6]:
# 2. EasyUQ

In [6]:
fit = idr(y_train, pd.DataFrame({'X': hres_train}))
idr_train = fit.predict()
idr_test = fit.predict(pd.DataFrame({'X': hres_test}))
print('CRPS of EasyUQ:', np.round(np.mean(idr_test.crps(y_test)), 3))

CRPS of EasyUQ: 0.732


In [ ]:
# 3. Single Gaussian

In [7]:
sigma = log_norm_optim(y_train, hres_train)
print('CRPS of SG:',np.round(np.mean(ps.crps_gaussian(y_test, mu=hres_test, sig=sigma)), 3))
print('CRPS of censored SG:',np.round(np.mean(crps_censored_gaussian(y_test, hres_test, sigma)), 3))

CRPS of SG: 1.244
CRPS of censored SG: 1.013


In [1]:
# 4. Ensemble data for lead time 1

In [3]:
# Get data
dim_train = 2896
dim_test = 721
rain = isodisreg.load_rain()
varNames = rain.columns[3:55]

ensemble = rain[varNames].to_numpy()
y = rain['obs'].to_numpy()

ensemble_train = ensemble[:dim_train, ]
ensemble_test = ensemble[dim_train:]
y_train = y[:dim_train]
y_test = y[dim_train:]

# CRPS of ensemble
crps_ens = np.mean(ps.crps_ensemble(y_test, ensemble_test))
print('CRPS of ensemble: %f' %crps_ens)

CRPS of ensemble: 0.752237


In [4]:
from crpsmixture import smooth_crps_ens
from helper_functions import ensemble_smoothing, llscore_ens_smoothing
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ll_train, h, df = ensemble_smoothing(ensemble_train, y_train)
ll = llscore_ens_smoothing(ensemble_test, y_test, h, df)
crpss = smooth_crps_ens(ensemble_test, y_test, h, df)
print('Fitted df = %d' %df , 'and h = %f' %h)
print('CRPS of smooth ensemble: %f' %crpss)
print('log score: %f' %ll)